In [2]:
%reset -f
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sqlite3 as sql
from pathlib import Path

def con_sql(db_name, dialect=None):
	db_file = Path(db_name)
	if not db_file.is_file():
		print("no such database file: {}".format(db_name))
		return
	if dialect is None:
		return sql.connect(db_name)
	return sql.connect(db_name, dialect)

conn = con_sql("database.sqlite")
c = conn.cursor() 

def describe_db(cur=None):
	import sqlite3 as sql
	import re
	try:
		c.execute("""SELECT name FROM sqlite_master WHERE type='table'""")
	except:
		return "db connection must be initialized before this function"
	if not isinstance(cur, sql.Cursor):
		return "expector cursor got {}".format(cur)
	tables = pd.DataFrame(c.fetchall())
	print("tables:", tables)
	for table in tables.values:
		print("table {} fields:".format(table))
		cur.execute("""PRAGMA table_info({})""".format(table[0]))
		table_df = pd.DataFrame(c.fetchall())
		print(table_df)
# place your cursor 'c' inside the function below
describe_db(c)

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


tables:                   0
0   sqlite_sequence
1           Matches
2  Teams_in_Matches
3             Teams
4      Unique_Teams
table ['sqlite_sequence'] fields:
   0     1 2  3     4  5
0  0  name    0  None  0
1  1   seq    0  None  0
table ['Matches'] fields:
   0         1        2  3     4  5
0  0  Match_ID  INTEGER  1  None  1
1  1       Div     TEXT  0  None  0
2  2    Season      INT  0  None  0
3  3      Date     TEXT  0  None  0
4  4  HomeTeam     TEXT  0  None  0
5  5  AwayTeam     TEXT  0  None  0
6  6      FTHG      INT  0  None  0
7  7      FTAG      INT  0  None  0
8  8       FTR     TEXT  0  None  0
table ['Teams_in_Matches'] fields:
   0               1    2  3     4  5
0  0        Match_ID  INT  0  None  0
1  1  Unique_Team_ID  INT  0  None  0
table ['Teams'] fields:
   0                       1     2  3     4  5
0  0                  Season  TEXT  0  None  0
1  1                TeamName  TEXT  0  None  0
2  2               KaderHome  TEXT  0  None  0
3  3            

In [101]:
team = 'Bochum'
q=("""
SELECT FTHG, FTAG, FTR, HomeTeam, AwayTeam, Season  
FROM Matches
WHERE ((Season=2011) AND (HomeTeam='{}')) AND (FTR='H') OR ((Season=2011) AND (AwayTeam='{}')) AND (FTR='A')
""".format(team,team))

df = pd.read_sql_query(q, conn)
return df.shape[0]

10

In [70]:
def get_team_list():
    q=("""
    SELECT TeamName  
    FROM Unique_Teams u
    JOIN Matches m
    WHERE (u.TeamName=m.HomeTeam) OR (u.TeamName=m.AwayTeam) AND (m.Season=2011)
    """)
    df = pd.read_sql_query(q, conn)
    df.drop_duplicates('TeamName', inplace=True)
    team_name_list = list(df.TeamName)
    
    return team_name_list
    
get_team_list()



['Bayern Munich',
 'Dortmund',
 'Leverkusen',
 'RB Leipzig',
 'Schalke 04',
 "M'gladbach",
 'Wolfsburg',
 'FC Koln',
 'Hoffenheim',
 'Hertha',
 'Mainz',
 'Hamburg',
 'Werder Bremen',
 'Ein Frankfurt',
 'Augsburg',
 'Freiburg',
 'Stuttgart',
 'Hannover',
 'Ingolstadt',
 'Darmstadt',
 'Paderborn',
 'Nurnberg',
 'Braunschweig',
 'Greuther Furth',
 'Fortuna Dusseldorf',
 'Kaiserslautern',
 'St Pauli',
 'Bochum',
 'Karlsruhe',
 'Bielefeld',
 'Cottbus',
 'Duisburg',
 'Hansa Rostock',
 'Aachen',
 'Union Berlin',
 'Heidenheim',
 'Dresden',
 'Sandhausen',
 'Erzgebirge Aue',
 'Kiel',
 'Regensburg',
 'Munich 1860',
 'Wurzburger Kickers',
 'Frankfurt FSV',
 'Aalen',
 'Oberhausen',
 'Osnabruck',
 'Koblenz',
 'Ahlen',
 'Wehen',
 'CZ Jena',
 'Offenbach',
 'Burghausen',
 'Unterhaching',
 'Saarbrucken',
 'Siegen',
 'Babelsberg',
 'Chemnitz',
 'Ein Trier',
 'Erfurt',
 'F Koln',
 'Gutersloh',
 'Homburg',
 'Lubeck',
 'Mannheim',
 'Meppen',
 'Oldenburg',
 'Reutlingen',
 'Schweinfurt',
 'Stuttgarter K',
 'T

In [179]:
def won_on_date(team,date):
    tup_list = game_dates_and_results_list(team)

    for tup in tup_list:
        if tup[0]==date:
            if tup[1]=='W':
                return 1
            else:
                return 0
            
won_on_date('Man United', '2011-09-18')

1

In [178]:
def game_dates_and_results_list(team):
    
    q=("""
    SELECT Date, FTR, HomeTeam, AwayTeam, Season
    FROM Matches 
    WHERE (Season=2011) AND ((HomeTeam='{}') OR (AwayTeam='{}'))
    ORDER BY Date
    """.format(team, team))
    
    
    df = pd.read_sql_query(q, conn)
    date_result_tuple_list = []
    for i, date in enumerate(df.Date):
        if df.FTR[i]=='H' and df.HomeTeam[i]==team:
            date_result_tuple_list.append((df.Date[i], 'W'))
        elif df.FTR[i]=='H' and df.HomeTeam[i]!=team:
            date_result_tuple_list.append((df.Date[i], 'L'))
        elif df.FTR[i]=='A' and df.AwayTeam[i]==team:
            date_result_tuple_list.append((df.Date[i], 'W'))
        elif df.FTR[i]=='A' and df.AwayTeam[i]!=team:
            date_result_tuple_list.append((df.Date[i], 'L'))
        else:
            date_result_tuple_list.append((df.Date[i], 'D'))
   
    return date_result_tuple_list

game_dates_list('Man United')



[('2011-08-14', 'W'),
 ('2011-08-22', 'W'),
 ('2011-08-28', 'W'),
 ('2011-09-10', 'W'),
 ('2011-09-18', 'W'),
 ('2011-09-24', 'D'),
 ('2011-10-01', 'W'),
 ('2011-10-15', 'D'),
 ('2011-10-23', 'L'),
 ('2011-10-29', 'W'),
 ('2011-11-05', 'W'),
 ('2011-11-19', 'W'),
 ('2011-11-26', 'D'),
 ('2011-12-03', 'W'),
 ('2011-12-10', 'W'),
 ('2011-12-18', 'W'),
 ('2011-12-21', 'W'),
 ('2011-12-26', 'W'),
 ('2011-12-31', 'L'),
 ('2012-01-04', 'L'),
 ('2012-01-14', 'W'),
 ('2012-01-22', 'W'),
 ('2012-01-31', 'W'),
 ('2012-02-05', 'D'),
 ('2012-02-11', 'W'),
 ('2012-02-26', 'W'),
 ('2012-03-04', 'W'),
 ('2012-03-11', 'W'),
 ('2012-03-18', 'W'),
 ('2012-03-26', 'W'),
 ('2012-04-02', 'W'),
 ('2012-04-08', 'W'),
 ('2012-04-11', 'L'),
 ('2012-04-15', 'W'),
 ('2012-04-22', 'D'),
 ('2012-04-30', 'L'),
 ('2012-05-06', 'W'),
 ('2012-05-13', 'W')]

In [123]:
def won_or_lost(team, date):
    q=("""
    SELECT Date, FTR, HomeTeam, AwayTeam, Season  
    FROM Matches
    WHERE ((Season=2011) AND (HomeTeam='{}') AND (Date='{}')) 
    AND (FTR='H') OR ((Season=2011) AND (AwayTeam='{}') AND (Date='{}')) AND (FTR='A')
    """.format(team,date,team,date))

    df = pd.read_sql_query(q, conn)
    return df

won_or_lost("Man United", '2012-04-08')

,Date,FTR,HomeTeam,AwayTeam,Season
0,2012-04-08,H,Man United,QPR,2011


In [105]:
def goals_scored(team):
    q=("""
    SELECT FTHG, FTAG
    FROM Matches
    WHERE (Season=2011) AND ((HomeTeam='{}') OR (AwayTeam='{}'))
    """.format(team,team))

    df = pd.read_sql_query(q, conn)
    total_goals = df.FTHG.sum() + df.FTAG.sum()
    return total_goals

goals_scored("Man United")

122

In [180]:
def num_wins(team):
    q=("""
    SELECT FTR, HomeTeam, AwayTeam, Season  
    FROM Matches
    WHERE ((Season=2011) AND (HomeTeam='{}')) AND (FTR='H') OR ((Season=2011) AND (AwayTeam='{}')) AND (FTR='A')
    """.format(team,team))

    wins = pd.read_sql_query(q, conn)
    return wins.shape[0]

num_wins('Man United')



28

In [181]:
def num_losses(team):
    q=("""
    SELECT FTHG, FTAG, FTR, HomeTeam, AwayTeam, Season  
    FROM Matches
    WHERE ((Season=2011) AND (HomeTeam='{}')) AND (FTR='A') OR ((Season=2011) AND (AwayTeam='{}')) AND (FTR='H')
    """.format(team,team))

    losses = pd.read_sql_query(q, conn)
    return losses.shape[0]

num_losses('Man United')

5

In [182]:
def num_draws(team):
    q=("""
    SELECT FTHG, FTAG, FTR, HomeTeam, AwayTeam, Season  
    FROM Matches
    WHERE ((Season=2011) AND (HomeTeam='{}')) AND (FTR='D') OR ((Season=2011) AND (AwayTeam='{}')) AND (FTR='D')
    """.format(team,team))

    draws = pd.read_sql_query(q, conn)
    return draws.shape[0]

num_draws('Man United')

5

In [185]:
def set_record(team):
    #This finds the wins
     q=("""
    SELECT FTR, HomeTeam, AwayTeam, Season  
    FROM Matches
    WHERE ((Season=2011) AND (HomeTeam='{}')) AND (FTR='H') OR ((Season=2011) AND (AwayTeam='{}')) AND (FTR='A')
    """.format(team,team))

    wins = pd.read_sql_query(q, conn)
    num_wins = wins.shape[0]

    #This finds the losses
    q=("""
    SELECT FTHG, FTAG, FTR, HomeTeam, AwayTeam, Season  
    FROM Matches
    WHERE ((Season=2011) AND (HomeTeam='{}')) AND (FTR='A') OR ((Season=2011) AND (AwayTeam='{}')) AND (FTR='H')
    """.format(team,team))

    losses = pd.read_sql_query(q, conn)
    num_losses = losses.shape[0]

    #This finds the drawws
     q=("""
    SELECT FTHG, FTAG, FTR, HomeTeam, AwayTeam, Season  
    FROM Matches
    WHERE ((Season=2011) AND (HomeTeam='{}')) AND (FTR='D') OR ((Season=2011) AND (AwayTeam='{}')) AND (FTR='D')
    """.format(team,team))

    draws = pd.read_sql_query(q, conn)
    num_draws = draws.shape[0]
    
    return [num_wins,num_losses,num_draws]

set_record('Man United')

[28, 5, 5]

In [3]:
q=("""
SELECT Date, Season 
FROM Matches
WHERE Season = 2011
ORDER BY Date
""")
pd.read_sql_query(q, conn)




,Date,Season
0,2011-07-15,2011
1,2011-07-15,2011
2,2011-07-15,2011
3,2011-07-16,2011
4,2011-07-16,2011
5,2011-07-17,2011
6,2011-07-17,2011
7,2011-07-17,2011
8,2011-07-18,2011
9,2011-07-22,2011
